In [ ]:
from tensorflow.keras import applications
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.keras import Sequential, Model 
from tensorflow.keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from tensorflow.keras import backend as k 
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

# Big dataset ~ 7000 images for 2 classes

In [ ]:
img_width, img_height = 224, 224
train_data_dir = "image/training_set"
validation_data_dir = "image/test_set"

In [ ]:
model = applications.VGG16(weights='imagenet',
                           include_top=False, #tắt chức năng classification để tự thêm vào sau
                           input_shape=(img_width, img_height,3))

58900480/58889256 [==============================] - 1s 0us/step


In [ ]:
model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

#Dataset nhiều hơn (big dataset)

In [ ]:
#Đóng băng VGG16 layer.
for layer in model.layers[:6]:
  layer.trainable = False

In [ ]:
#thêm các layers tùy biến (ANN)
x = model.output #lấy kết quả của model
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(2, activation='sigmoid')(x)

In [ ]:
#creating the final model
model_final = Model(inputs = model.input, outputs=predictions)

In [ ]:
#compile the model
model_final.compile(loss='categorical_crossentropy',
                    optimizer='adam',
                    metrics=['accuracy'])

In [ ]:
# Initiate the train and test generators with data Augumentation 
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   horizontal_flip = True,
                                   fill_mode = "nearest",
                                   zoom_range = 0.3,
                                   width_shift_range = 0.3,
                                   height_shift_range=0.3,
                                   rotation_range=30)

test_datagen = ImageDataGenerator(rescale = 1./255)


In [ ]:
train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                    target_size = (img_height, 
                                                                   img_width),
                                                    batch_size = 32, 
                                                    class_mode = "categorical")

validation_generator = test_datagen.flow_from_directory(validation_data_dir,
                                                    target_size = (img_height, 
                                                                   img_width),
                                                    class_mode = "categorical")


Found 5937 images belonging to 2 classes.
Found 1340 images belonging to 2 classes.


In [ ]:
# Save the model according to the conditions  
checkpoint = ModelCheckpoint("owl_and_eagle_big_vgg16.h5", 
                             monitor='val_loss', 
                             verbose=1, 
                             save_best_only=True, 
                             save_weights_only=False, 
                             mode='auto', 
                             save_freq=1)
early = EarlyStopping(monitor='val_loss', 
                      min_delta=0.001, 
                      patience=10, 
                      verbose=1, 
                      mode='auto')


In [ ]:
import datetime

In [ ]:
t0 = datetime.datetime.now()
print(t0)

2021-11-08 15:10:27.036584


In [ ]:
#train the model
history = model_final.fit(train_generator,
                          epochs=100,
                          validation_data = validation_generator,
                          verbose=1,
                          callbacks = [checkpoint, early])

Epoch 1/100
186/186 [==============================] - 3778s 20s/step - loss: 1.3674 - accuracy: 0.4969 - val_loss: 0.6928 - val_accuracy: 0.5201
Epoch 2/100
186/186 [==============================] - 140s 753ms/step - loss: 0.6931 - accuracy: 0.5011 - val_loss: 0.6924 - val_accuracy: 0.5201
Epoch 3/100
186/186 [==============================] - 141s 757ms/step - loss: 0.6933 - accuracy: 0.5031 - val_loss: 0.6931 - val_accuracy: 0.5201
Epoch 4/100
186/186 [==============================] - 142s 761ms/step - loss: 0.6932 - accuracy: 0.5011 - val_loss: 0.6931 - val_accuracy: 0.5201
Epoch 5/100
186/186 [==============================] - 143s 766ms/step - loss: 0.6933 - accuracy: 0.5075 - val_loss: 0.6930 - val_accuracy: 0.5201
Epoch 6/100
186/186 [==============================] - 144s 770ms/step - loss: 0.6933 - accuracy: 0.5050 - val_loss: 0.6926 - val_accuracy: 0.5201
Epoch 7/100
186/186 [==============================] - 145s 778ms/step - loss: 0.6933 - accuracy: 0.5063 - val_loss: 0.

In [ ]:
t1 = datetime.datetime.now()
print(t1)

2021-11-08 16:38:34.594510


In [ ]:
print(t1 - t0)

1:28:07.557926


In [ ]:
import pandas as pd

In [ ]:
history_df = pd.DataFrame(history.history)
history_df.tail()

,loss,accuracy,val_loss,val_accuracy
6,0.693272,0.506316,0.693841,0.479851
7,0.693413,0.496210,0.693216,0.479851
8,0.692887,0.512212,0.692353,0.520149
9,0.693385,0.497726,0.692530,0.520149
10,0.693236,0.504127,0.692736,0.520149


In [ ]:
#save model
from tensorflow.keras.models import load_model
model_final.save('owl_and_eagle_big_vgg16_final.h5')
print('save!!!')

save!!!


#Nhận xét

Với 3 model (sử dụng model tự build, vgg16 với small dataset, vgg16 với big dataset), ta có nhận xét sau
* Model tự build: time ~ 53 phút, accuracy ~ 83%
* Vgg16 với small dataset: time ~ 7 phút, accuracy ~ 70%
* Vgg16 với big dataset: time ~ 1h28p, accuracy ~ 52%
--> sử dụng model tự build (câu 3) cho kết quả tốt hơn và thời gian nhanh hơn.
